# Project: Soccer Data Analysis using Python & SQL

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

> This soccer database comes from Kaggle. It contains data of soccer matches, players, and teams from several European countries from 2008 to 2016. This dataset is quite extensive, it contains **25,000 plus matches of 10,000 players in 11 European Countries with their lead championship for Seasons 2008 to 2016**. Players and Teams' attributes* are sourced from EA Sports' FIFA video game series, including the weekly updates. Team line up with squad formation (X, Y coordinates) .Betting odds from up to 10 providers. Detailed match events (goal types, possession, corner, cross, fouls, cards etc…) for +10,000 matches are included.

## Question 


1. Which players had the most penalties? 

2. What teams improved the most over the time period?

3. What team attributes lead to the most victories?

4. Which player has improved most over time?

Try to build your report around the analysis of at least one dependent variable and three independent variables.



In [1]:
#Use this cell to set up import statements for all of the packages that you
#plan to use.

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_sql)
import sqlite3 #
import matplotlib.pyplot as plt

In [2]:
#database files are available in the "../Soccer/" directory.

path = "D:/Data Analysis/Data/Soccer/"  #path where the sqlite database file resides
database = path + 'database.sqlite' #the database file itself

First we will create the connection to the DB and see what tables we have.

The basic structure of the query is very simple: You define what you want to see after the **SELECT** statement, **Asterisk** means all possible columns, You choose the table after the **FROM**, You add the conditions for the data you want to use from the table(s) after the **WHERE**.

The structure, and the order of the sections matter, while spaces, new lines, capital words and indentation are there to make the code easier to read.


In [3]:
conn = sqlite3.connect(database) #connection object that represents the database

#query inside a cursor object
tables = pd.read_sql("""SELECT *
                          FROM sqlite_master
                         WHERE type='table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


<a id='wrangling'></a>
## Data Wrangling

> **Tip**: In this section of the report, you will load in the data, check for cleanliness, and then trim and clean your dataset for analysis. Make sure that you document your steps carefully and justify your cleaning decisions.

### General Properties

In [18]:
player_attributes = pd.read_sql("""SELECT *
                                     FROM Player_Attributes;""", conn)
player_attributes

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183973,183974,102359,39902,2009-08-30 00:00:00,83.0,85.0,right,medium,low,84.0,...,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183974,183975,102359,39902,2009-02-22 00:00:00,78.0,80.0,right,medium,low,74.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183975,183976,102359,39902,2008-08-30 00:00:00,77.0,80.0,right,medium,low,74.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,183977,102359,39902,2007-08-30 00:00:00,78.0,81.0,right,medium,low,74.0,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


In [23]:
player_attributes.describe()

,id,player_fifa_api_id,player_api_id,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
count,183978.00000,183978.000000,183978.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000,181265.000000,...,181265.000000,183142.000000,183142.000000,183142.000000,181265.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000
mean,91989.50000,165671.524291,135900.617324,68.600015,73.460353,55.086883,49.921078,57.266023,62.429672,49.468436,...,57.873550,55.003986,46.772242,50.351257,48.001462,14.704393,16.063612,20.998362,16.132154,16.441439
std,53110.01825,53851.094769,136927.840510,7.041139,6.592271,17.242135,19.038705,16.488905,14.194068,18.256618,...,15.144086,15.546519,21.227667,21.483706,21.598778,16.865467,15.867382,21.452980,16.099175,17.198155
min,1.00000,2.000000,2625.000000,33.000000,39.000000,1.000000,1.000000,1.000000,3.000000,1.000000,...,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,45995.25000,155798.000000,34763.000000,64.000000,69.000000,45.000000,34.000000,49.000000,57.000000,35.000000,...,49.000000,45.000000,25.000000,29.000000,25.000000,7.000000,8.000000,8.000000,8.000000,8.000000
50%,91989.50000,183488.000000,77741.000000,69.000000,74.000000,59.000000,53.000000,60.000000,65.000000,52.000000,...,60.000000,57.000000,50.000000,56.000000,53.000000,10.000000,11.000000,12.000000,11.000000,11.000000
75%,137983.75000,199848.000000,191080.000000,73.000000,78.000000,68.000000,65.000000,68.000000,72.000000,64.000000,...,69.000000,67.000000,66.000000,69.000000,67.000000,13.000000,15.000000,15.000000,15.000000,15.000000
max,183978.00000,234141.000000,750584.000000,94.000000,97.000000,95.000000,97.000000,98.000000,97.000000,93.000000,...,97.000000,96.000000,96.000000,95.000000,95.000000,94.000000,93.000000,97.000000,96.000000,96.000000


In [19]:
player = pd.read_sql("""SELECT *
                          FROM  Player;""", conn)
player

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154
...,...,...,...,...,...,...,...
11055,11071,26357,Zoumana Camara,2488,1979-04-03 00:00:00,182.88,168
11056,11072,111182,Zsolt Laczko,164680,1986-12-18 00:00:00,182.88,176
11057,11073,36491,Zsolt Low,111191,1979-04-29 00:00:00,180.34,154
11058,11074,35506,Zurab Khizanishvili,47058,1981-10-06 00:00:00,185.42,172


In [20]:
macth = pd.read_sql("""SELECT *
                         FROM Match;""", conn)
macth

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
league = pd.read_sql("""SELECT *
                          FROM League;""", conn)
league

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A
5,13274,13274,Netherlands Eredivisie
6,15722,15722,Poland Ekstraklasa
7,17642,17642,Portugal Liga ZON Sagres
8,19694,19694,Scotland Premier League
9,21518,21518,Spain LIGA BBVA


In [14]:
country = pd.read_sql("""SELECT *
                           FROM  Country;""", conn)
country

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


In [16]:
team = pd.read_sql("""SELECT *
                           FROM Team;""", conn)
team

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB
...,...,...,...,...,...
294,49479,10190,898.0,FC St. Gallen,GAL
295,49837,10191,1715.0,FC Thun,THU
296,50201,9777,324.0,Servette FC,SER
297,50204,7730,1862.0,FC Lausanne-Sports,LAU


In [17]:
team_attributes = pd.read_sql("""SELECT *
                           FROM  Team_Attributes;""", conn)
team_attributes

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10 00:00:00,47,Balanced,41.0,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22 00:00:00,70,Fast,NaN,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22 00:00:00,47,Balanced,NaN,Little,52,Mixed,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1453,1454,15005,10000,2011-02-22 00:00:00,52,Balanced,NaN,Little,52,Mixed,...,53,Normal,Organised,46,Medium,48,Press,53,Normal,Cover
1454,1455,15005,10000,2012-02-22 00:00:00,54,Balanced,NaN,Little,51,Mixed,...,50,Normal,Organised,44,Medium,55,Press,53,Normal,Cover
1455,1456,15005,10000,2013-09-20 00:00:00,54,Balanced,NaN,Little,51,Mixed,...,32,Little,Organised,44,Medium,58,Press,37,Normal,Cover
1456,1457,15005,10000,2014-09-19 00:00:00,54,Balanced,42.0,Normal,51,Mixed,...,32,Little,Organised,44,Medium,58,Press,37,Normal,Cover


In [4]:
finiding_null = pd.read_sql("""SELECT *
                                FROM  Match
                                LIMIT 100;""", conn)
finiding_null

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,1,1,2008/2009,19,2009-01-24 00:00:00,492810,4049,9994,2,...,2.30,2.70,3.20,2.35,2.75,3.20,2.35,2.80,3.20,2.25
96,97,1,1,2008/2009,19,2009-01-24 00:00:00,492814,8203,9996,2,...,3.00,2.30,3.20,2.80,2.30,3.20,2.85,2.30,3.20,2.75
97,98,1,1,2008/2009,19,2009-01-24 00:00:00,492817,9984,8571,0,...,5.50,1.55,3.50,5.50,1.57,3.50,5.50,1.53,3.60,5.50
98,99,1,1,2008/2009,19,2009-01-24 00:00:00,492819,7947,9993,0,...,2.25,2.90,3.20,2.25,2.85,3.20,2.30,2.88,3.20,2.20


In [5]:
# Load your data and print out a few lines. Perform operations to inspect data

#Team with top 10 buildUpPlaySpeed
best_buildupplayspeed = pd.read_sql("""SELECT t.team_long_name,
                                              t.team_api_id,
                                              ta.buildUpPlaySpeed,
                                              ta.buildupplayspeedclass
                                         FROM Team_Attributes ta
                                         JOIN Team t
                                           ON ta.team_api_id = t.team_api_id
                                        ORDER BY 3 DESC
                                        LIMIT 10;""", conn)
best_buildupplayspeed

,team_long_name,team_api_id,buildUpPlaySpeed,buildUpPlaySpeedClass
0,Carpi,208931,80,Fast
1,Hannover 96,9904,78,Fast
2,Udinese,8600,78,Fast
3,West Ham United,8654,77,Fast
4,Borussia Dortmund,9789,76,Fast
5,Borussia Mönchengladbach,9788,76,Fast
6,Torino,9804,76,Fast
7,Arsenal,9825,75,Fast
8,Catania,8530,75,Fast
9,TSG 1899 Hoffenheim,8226,75,Fast


In [6]:
player = pd.read_sql("""SELECT *
                          FROM Player
                      
                         ;""", conn)
player

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154
...,...,...,...,...,...,...,...
11055,11071,26357,Zoumana Camara,2488,1979-04-03 00:00:00,182.88,168
11056,11072,111182,Zsolt Laczko,164680,1986-12-18 00:00:00,182.88,176
11057,11073,36491,Zsolt Low,111191,1979-04-29 00:00:00,180.34,154
11058,11074,35506,Zurab Khizanishvili,47058,1981-10-06 00:00:00,185.42,172


In [7]:
#checking weather ther is a duplicate entry for players
duplicate_player = pd.read_sql("""SELECT player_api_id,
                                         player_name,
                                         player_fifa_api_id,
                                         COUNT(*) player_count
                                    FROM Player
                                   GROUP BY 1,2,3
                                  HAVING player_count > 1;""", conn)
duplicate_player

,player_api_id,player_name,player_fifa_api_id,player_count


In [8]:
matches = pd.read_sql("""SELECT *
                          FROM Match
                      WHERE match_api_id IS NULL
                         ;""", conn)
matches

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA


In [9]:
WITH cte AS (SELECT contact_id, 
                    first_name, 
                    last_name, 
                    email, 
        ROW_NUMBER() OVER (PARTITION BY first_name, 
                                        last_name, 
                                        email
                              ORDER BY 
                                    first_name, 
                                    last_name, 
                                    email) row_num
              FROM sales.contacts)
DELETE FROM cte
 WHERE row_num > 1;

SyntaxError: invalid syntax (<ipython-input-9-4258596581d5>, line 1)

In [ ]:
#player with player attributes of 2015-16 season 
player_with_attributes = pd.read_sql("""SELECT p.player_name player_name,
                                    p.player_api_id,
                                    pa.overall_rating best_rating,
                                    pa.date,
                                    pa.potential,
                                    pa.finishing finishing,
                                    pa.short_passing,
                                    pa.volleys,
                                    pa.dribbling,
                                    pa.free_kick_accuracy,
                                    pa.penalties
                               FROM player p
                               JOIN player_attributes pa
                                 ON p.player_api_id = pa.player_api_id
                              WHERE date BETWEEN '2015-01-01' AND '2016-12-31';""", conn)
player_with_attributes

In [ ]:
#player with player attributes of 2015-16 season with descending order of player_rating
best_player_rating = pd.read_sql("""WITH t1 AS (SELECT p.player_name player_name,
                                                       p.player_api_id,
                                                       pa.overall_rating best_rating,
                                                       pa.date,
                                                       pa.potential,
                                                       pa.finishing finishing,
                                                       pa.short_passing,
                                                       pa.volleys,
                                                       pa.dribbling,
                                                       pa.free_kick_accuracy,
                                                       pa.penalties
                                                  FROM player p
                                                  JOIN player_attributes pa
                                                    ON p.player_api_id = pa.player_api_id
                                                 WHERE date BETWEEN '2015-01-01' AND '2016-12-31')
                                  SELECT t1.player_name name_of_the_player, 
                                         t1.player_api_id,
                                         t1.best_rating player_rating,
                                         t1.finishing player_finishing,
                                         t1.penalties
                                    FROM t1
                                   ORDER BY 3 DESC;""",conn)
best_player_rating

In [ ]:
best_players_of_2015 = pd.read_sql("""WITH t2 AS (WITH t1 AS (SELECT p.player_name player_name,
                                                                     p.player_api_id,
                                                                     pa.overall_rating best_rating,
                                                                     pa.date,
                                                                     pa.potential,
                                                                     pa.finishing finishing,
                                                                     pa.short_passing,
                                                                     pa.volleys,
                                                                     pa.dribbling,
                                                                     pa.free_kick_accuracy,
                                                                     pa.penalties
                                                                FROM player p
                                                                JOIN player_attributes pa
                                                                  ON p.player_api_id = pa.player_api_id
                                                               WHERE date BETWEEN '2015-01-01' AND '2015-12-31')
                                                      
                                                SELECT t1.player_name name_of_the_player, 
                                                       t1.player_api_id,
                                                       t1.best_rating player_rating,
                                                       t1.finishing player_finishing,
                                                       t1.penalties
                                                  FROM t1
                                                 ORDER BY 3 DESC)

                                      SELECT DISTINCT t2.name_of_the_player,
                                                      t2.player_api_id,
                                             MAX(t2.player_rating) rating
                                        FROM t2
                                       ORDER BY 3 DESC
                                       LIMIT 10;""", conn)
best_players_of_2015

In [ ]:
SELECT t2.name_of_the_player,
                                  t2.player_rating,
                                  t2.player_finishing
                                  COUNT(*)
                             FROM t2
                             GROUP BY t2.player_rating, t2.player_finishing
                             HAVING COUNT(t2.name_of_the_player > 1)

In [ ]:
#types and look for instances of missing or possibly errant data.

In [ ]:
#lets have alook into the match table

match = pd.read_sql("""SELECT *
                         FROM Match
                        LIMIT 100;""", conn)
match

In [ ]:
leagues = pd.read_sql("""SELECT *
                           FROM League l
                           JOIN Country c
                             ON c.id = l.country_id;""", conn)
leagues

### List of teams

ORDER BY defines the sorting of the output - ascending or descending (DESC)

LIMIT, limits the number of rows in the output - after the sorting


In [ ]:
teams = pd.read_sql("""SELECT *
                         FROM Team
                        ORDER BY team_long_name
                        LIMIT 10;""", conn)
teams

### List of matches

In this exapmle we will show only the columns that interests us, so instead of * we will use the exact names.

Some of the cells have the same name (Country.name,League.name). We will rename them using AS.

As you can see, this query has much more joins. The reasons is because the DB is designed in a star structure - one table (Match) with all the "performance" and metrics, but only keys and IDs, while all the descriptive information stored in other tables (Country, League, Team)

Note that Team is joined twice. This is a tricky one, as while we are using the same table name, we basically bring two different copies (and rename them using AS). The reason is that we need to bring information about two different values (home_team_api_id, away_team_api_id), and if we join them to the same table, it would mean that they are equal to each other.

You will also note that the Team tables are joined using left join. The reason is decided that I would prefer to keep the matches in the output - even if on of the teams doesn't appear in the Team table.

ORDER defines the order of the output, and comes before the LIMIT and after the WHERE


In [ ]:
detailed_matches = pd.read_sql("""SELECT m.id, 
                                         c.name AS country_name, 
                                         l.name AS league_name, 
                                         season, 
                                         stage, 
                                         date,
                                         HT.team_long_name AS  home_team,
                                         AT.team_long_name AS away_team,
                                         home_team_goal, 
                                         away_team_goal                                        
                                    FROM Match m
                                    JOIN Country c 
                                      ON c.id = m.country_id
                                    JOIN League l
                                      ON l.id = m.league_id
                                    LEFT JOIN Team AS HT 
                                      ON HT.team_api_id = m.home_team_api_id
                                    LEFT JOIN Team AS AT 
                                      ON AT.team_api_id = m.away_team_api_id
                                   WHERE country_name = 'Spain'
                                   ORDER by date
                                   LIMIT 10;""", conn)
detailed_matches

> **Tip**: You should _not_ perform too many operations in each cell. Create cells freely to explore your data. One option that you can take with this project is to do a lot of explorations in an initial notebook. These don't have to be organized, but make sure you use enough comments to understand the purpose of each code cell. Then, after you're done with your analysis, create a duplicate notebook where you will trim the excess and organize your steps so that you have a flowing, cohesive report.

> **Tip**: Make sure that you keep your reader informed on the steps that you are taking in your investigation. Follow every code cell, or every set of related code cells, with a markdown cell to describe to the reader what was found in the preceding cell(s). Try to make it so that the reader can then understand what they will be seeing in the following cell(s).

### Data Cleaning (Replace this with more specific notes!)

In [ ]:
# After discussing the structure of the data and any problems that need to be
#   cleaned, perform those cleaning steps in the second part of this section.


### Let's do some basic analytics

Here we are starting to look at the data at more aggregated level. Instead of looking on the raw data we will start to grouping it to different levels we want to examine. In this example, we will base it on the previous query, remove the match and date information, and look at it at the country-league-season level.

The functionality we will use for that is GROUP BY, that comes between the WHERE and ORDER

Once you chose what level you want to analyse, we can devide the select statement to two:

    Dimensions - those are the values we describing, same that we group by later.
    Metrics - all the metrics have to be aggregated using functions.. The common functions are: sum(), count(), count(distinct), avg(), min(), max()

Note - it is very important to use the same dimensions both in the select, and in the GROUP BY. Otherwise the output might be wrong.

Another functionality that can be used after grouping, is HAVING. This adds another layer of filtering the data, this time the output of the table after the grouping. A lot of times it is used to clean the output.


In [ ]:
leages_by_season = pd.read_sql("""SELECT c.name AS country_name, 
                                         l.name AS league_name, 
                                         m.season,
                                         COUNT(distinct m.stage) AS number_of_stages,
                                         COUNT(distinct HT.team_long_name) AS number_of_teams,
                                         AVG(m.home_team_goal) AS avg_home_team_scors, 
                                         AVG(m.away_team_goal) AS avg_away_team_goals, 
                                         AVG(m.home_team_goal - m.away_team_goal) AS avg_goal_dif, 
                                         AVG(m.home_team_goal + m.away_team_goal) AS avg_goals, 
                                         SUM(m.home_team_goal + m.away_team_goal) AS total_goals                                       
                                    FROM Match m
                                    JOIN Country c
                                      ON c.id = m.country_id
                                    JOIN League l
                                      ON l.id = m.league_id
                                    LEFT JOIN Team AS HT 
                                      ON HT.team_api_id = m.home_team_api_id
                                    LEFT JOIN Team AS AT 
                                      ON AT.team_api_id = m.away_team_api_id
                                   WHERE country_name in ('Spain', 'Germany', 'France', 'Italy', 'England')
                                   GROUP BY c.name, l.name, m.season
                                  HAVING count(distinct m.stage) > 10
                                   ORDER BY c.name, l.name, m.season DESC;""", conn)
leages_by_season

In [ ]:
df = pd.DataFrame(index=np.sort(leages_by_season['season'].unique()), columns=leages_by_season['country_name'].unique())

df.loc[:,'Germany'] = list(leages_by_season.loc[leages_by_season['country_name']=='Germany','avg_goals'])
df.loc[:,'Spain']   = list(leages_by_season.loc[leages_by_season['country_name']=='Spain','avg_goals'])
df.loc[:,'France']   = list(leages_by_season.loc[leages_by_season['country_name']=='France','avg_goals'])
df.loc[:,'Italy']   = list(leages_by_season.loc[leages_by_season['country_name']=='Italy','avg_goals'])
df.loc[:,'England']   = list(leages_by_season.loc[leages_by_season['country_name']=='England','avg_goals'])

df.plot(figsize=(12,5),title='Average Goals per Game Over Time')

In [ ]:
df = pd.DataFrame(index=np.sort(leages_by_season['season'].unique()), columns=leages_by_season['country_name'].unique())

df.loc[:,'Germany'] = list(leages_by_season.loc[leages_by_season['country_name']=='Germany','avg_goal_dif'])
df.loc[:,'Spain']   = list(leages_by_season.loc[leages_by_season['country_name']=='Spain','avg_goal_dif'])
df.loc[:,'France']   = list(leages_by_season.loc[leages_by_season['country_name']=='France','avg_goal_dif'])
df.loc[:,'Italy']   = list(leages_by_season.loc[leages_by_season['country_name']=='Italy','avg_goal_dif'])
df.loc[:,'England']   = list(leages_by_season.loc[leages_by_season['country_name']=='England','avg_goal_dif'])

df.plot(figsize=(12,5),title='Average Goals Difference Home vs Out')

### Query Run Order

Now that we are familiar with most of the functionalities being used in a query, it is very important to understand the order that code runs.

First, order of how we write it (reminder):

    SELECT
    FROM
    JOIN
    WHERE
    GROUP BY
    HAVING
    ORDER BY
    LIMIT

Now, the actul order that things happens. First, you can think of this part as creating a new temporal table in the memory:

    Define which tables to use, and connect them (FROM + JOIN)
    Keep only the rows that apply to the conditions (WHERE)
    Group the data by the required level (if need) (GROUP BY)
    Choose what information you want to have in the new table. It can have just rawdata (if no grouping), or combination of dimensions (from the grouping), and metrics Now, you chose that to show from the table:
    Order the output of the new table (ORDER BY)
    Add more conditions that would filter the new created table (HAVING)
    Limit to number of rows - would cut it according the soring and the having filtering (LIMIT)



### Sub Queries and Functions

Using subqueries is an essential tool in SQL, as it allows manipulating the data in very advanced ways without the need of any external scripts, and especially important when your tables stractured in such a way that you can't be joined directly.

In our example, I'm trying to join between a table that holds player's basic details (name, height, weight), to a table that holds more attributes. The problem is that while the first table holds one row for each player, the key in the second table is player+season, so if we do a regular join, the result would be a Cartesian product, and each player's basic details would appear as many times as this player appears in the attributes table. The problem with of course is that the average would be skewed towards players that appear many times in the attribute table.

The solution, is to use a subquery. We would need to group the attributes table, to a different key - player level only (without season). Of course we would need to decide first how we would want to combine all the attributes to a single row. I used average, but one can also decide on maximum, latest season and etc. Once both tables have the same keys, we can join them together (think of the subquery as any other table, only temporal), knowing that we won't have duplicated rows after the join.

In addition, you can see here two examples of how to use functions:

    Conditional function is an important tool for data manipulation. While IF statement is very popular in other languages, SQLite is not supporting it, and it's implemented using CASE + WHEN + ELSE statement. As you can see, based on the input of the data, the query would return different results.

    ROUND - straight sorward. Every SQL languages comes with a lot of usefull functions by default.



In [ ]:
players_height = pd.read_sql("""SELECT CASE WHEN ROUND(height)<165 then 165
                                            WHEN ROUND(height)>195 then 195
                                            ELSE ROUND(height)
                                            END AS calc_height, 
                               COUNT (height) AS distribution, 
                                 AVG (PA_Grouped.avg_overall_rating) AS avg_overall_rating,
                                 AVG (PA_Grouped.avg_potential) AS avg_potential,
                                 AVG (weight) AS avg_weight 
                                FROM PLAYER
                                LEFT JOIN (SELECT Player_Attributes.player_api_id, 
                                              AVG (Player_Attributes.overall_rating) AS avg_overall_rating,
                                              AVG (Player_Attributes.potential) AS avg_potential  
                                             FROM Player_Attributes
                                            GROUP BY Player_Attributes.player_api_id) AS PA_Grouped 
                                 ON PLAYER.player_api_id = PA_Grouped.player_api_id
                              GROUP BY calc_height
                              ORDER BY calc_height;""", conn)
players_height


In [ ]:
players_height.plot(x=['calc_height'],y=['avg_overall_rating'],figsize=(12,5),title='Potential vs Height')

<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 Which players had the most penalties?

In [22]:
# Use this, and more code cells, to explore your data. Don't forget to add
total_penalties = pd.read_sql("""SELECT p.player_name P_name, 
                                        SUM(pa.penalties)
                                   FROM player_attributes pa
                                   JOIN player p
                                     ON p.id = pa.player_api_id
                                  GROUP BY 1
                                  ORDER BY 2 DESC
                                  LIMIT 10;""", conn)
total_penalties



#   Markdown cells to document your observations and findings.


,P_name,SUM(pa.penalties)
0,Edinho,2484
1,Mariusz Rybicki,2210
2,Maximiliano Pellegrino,2178
3,Fabio Quagliarella,2081
4,Jordan Pickford,1967
5,Tugay Kerimoglou,1909
6,Ricardo Oliveira,1879
7,Rai Vloet,1854
8,Filipe Augusto,1827
9,Franco Di Santo,1781


### Research Question 2  What team attributes lead to the most victories?

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


### Research Question 3 What teams improved the most over the time period?

In [32]:
top_players = pd.read_sql("""/* Tope 20 players of 2008 having higher overall_rating*/
SELECT pa.id,
       p.player_name,
	   pa.date,
       pa.overall_rating,
       pa.potential,
       pa.preferred_foot,
       pa.attacking_work_rate,
       pa.defensive_work_rate,
       pa.crossing,
       pa.finishing,
       pa.heading_accuracy,
       pa.short_passing,
       pa.volleys,
       pa.dribbling,
       pa.curve,
       pa.free_kick_accuracy,
       pa.long_passing,
       pa.ball_control,
       pa.acceleration
  FROM Player_Attributes pa
  JOIN Player p
    ON pa.player_api_id = p.id
 WHERE pa.date BETWEEN '2008-01-01' AND '2008-12-31'
 ORDER BY overall_rating DESC
 LIMIT 20;""", conn)
top_players

,id,player_name,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration
0,13283,Franco Di Santo,2008-08-30 00:00:00,84,91,right,low,low,83,85,58,90,77.0,91,77.0,65,78,92,89
1,8062,Franco Brienza,2008-08-30 00:00:00,79,90,right,high,medium,75,82,69,74,68.0,73,66.0,67,73,81,80
2,137868,Dudu,2008-08-30 00:00:00,78,86,right,medium,medium,78,79,63,82,74.0,80,75.0,84,78,80,80
3,40330,Emil Hallfredsson,2008-08-30 00:00:00,76,64,left,medium,medium,40,41,85,41,20.0,40,30.0,35,54,44,62
4,176599,Edinho,2008-08-30 00:00:00,76,81,left,high,medium,73,74,53,73,75.0,83,72.0,62,67,79,87
5,176600,Edinho,2008-02-22 00:00:00,76,84,left,high,medium,61,74,75,71,75.0,82,72.0,68,68,82,84
6,52931,Mariusz Rybicki,2008-08-30 00:00:00,75,82,left,medium,medium,78,67,72,79,64.0,76,70.0,66,70,77,72
7,91976,Jordan Pickford,2008-08-30 00:00:00,75,76,right,medium,low,78,64,63,74,66.0,78,79.0,82,76,74,78
8,44853,Tugay Kerimoglou,2008-08-30 00:00:00,74,84,right,medium,low,76,70,59,78,78.0,78,78.0,59,67,78,77
9,52556,Marko Pantelic,2008-08-30 00:00:00,74,79,left,None,_0,73,73,72,75,NaN,76,NaN,75,70,74,82


In [34]:
Teams = pd.read_sql("""SELECT *
                         FROM Team
                         WHERE team_short_name IS 'CHE';""", conn)
teams

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,16848,8350,29,1. FC Kaiserslautern,KAI
1,15624,8722,31,1. FC Köln,FCK
2,16239,8165,171,1. FC Nürnberg,NUR
3,16243,9905,169,1. FSV Mainz 05,MAI
4,11817,8576,614,AC Ajaccio,AJA
5,11074,108893,111989,AC Arles-Avignon,ARL
6,49116,6493,1714,AC Bellinzona,BEL
7,26560,10217,650,ADO Den Haag,HAA
8,9537,8583,57,AJ Auxerre,AUX
9,9547,9829,69,AS Monaco,MON


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!